In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [ ]:
books

In [ ]:
books["tagged_description"]

In [ ]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header=False)

In [ ]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

In [ ]:
documents[0]

In [ ]:
from openai import OpenAI
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

models = client.models.list()

for model in models:
    print(model.id)

In [ ]:
db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(model="text-embedding-ada-002", api_key=OPENAI_API_KEY))


In [ ]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query=query, k = 10)
docs

In [ ]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

In [ ]:
def retrieve_semantic_recommendation(
    query: str,
    top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)
    books_list = []
    
    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]
    return books[books["isbn13"].isin(books_list)].head(top_k)

In [ ]:
retrieve_semantic_recommendation("A book to teach children about nature")